# Reinforcement Learning - Deep Q Network

In [1]:
import numpy as np
import random
import torch
import copy
from torch import nn
import torch.nn.functional as F
from torch import optim
from tqdm import tqdm as _tqdm
import os
import pickle

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1) 

%matplotlib inline
import matplotlib.pyplot as plt
import sys
import time

# !pip install gym==0.10.8
# !pip install box2d

import gym
import pandas as pd


assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

In [2]:
class QNetwork(nn.Module):
    
    def __init__(self, num_hidden=128, observation_space=4, action_space=2):
        nn.Module.__init__(self)
        self.l1 = nn.Linear(observation_space, num_hidden)
        self.l2 = nn.Linear(num_hidden, action_space)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.l1(x))
        out = self.l2(out)

        return out

In [3]:
class ReplayMemory:
    
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, transition):
        self.memory.append(transition)
        if len(self.memory) > self.capacity:
            self.memory.pop(0)

    def sample(self, batch_size):
        batch = random.sample(self.memory, batch_size)
        return batch

    def __len__(self):
        return len(self.memory)

In [4]:
class EpsilonGreedyPolicy(object):
    """
    A simple epsilon greedy policy.
    """
    def __init__(self, Q, epsilon):
        self.Q = Q
        self.epsilon = epsilon
    
    def sample_action(self, obs, device):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            obs: current state

        Returns:
            An action (int).
        """
        if random.random() < self.epsilon:
            action = random.randint(0, 1)
        else:
            with torch.no_grad():
                action = torch.argmax(self.Q(torch.Tensor(obs).to(device))).item()
        return int(action)
        
    def set_epsilon(self, epsilon):
        self.epsilon = epsilon

In [5]:
def get_epsilon(it, start=1, end=0.05, length=1000):
    if it < length:
        epsilon =  - it/(1000/0.95)
    else:
        epsilon = 0.05
    return epsilon

def smooth(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / float(N)

def compute_q_vals(Q, states, actions, device):
    """
    This method returns Q values for given state action pairs.
    
    Args:
        Q: Q-net
        states: a tensor of states. Shape: batch_size x obs_dim
        actions: a tensor of actions. Shape: Shape: batch_size x 1

    Returns:
        A torch tensor filled with Q values. Shape: batch_size x 1.
    """
    QValues = Q(states.to(device))[torch.arange(len(actions)).to(device), actions.squeeze().to(device)]
    return QValues.unsqueeze(dim=1)

def compute_targets(Q, rewards, next_states, dones, discount_factor, target_net, device):
    """
    This method returns targets (values towards which Q-values should move).
    
    Args:
        Q: Q-net
        rewards: a tensor of actions. Shape: Shape: batch_size x 1
        next_states: a tensor of states. Shape: batch_size x obs_dim
        dones: a tensor of boolean done flags (indicates if next_state is terminal) Shape: batch_size x 1
        discount_factor: discount
    Returns:
        A torch tensor filled with target values. Shape: batch_size x 1.
    """
    maxQs = torch.max(target_net(next_states.to(device)), 1).values
    if isinstance(dones, torch.BoolTensor):
        targets = rewards.to(device) + discount_factor * maxQs.unsqueeze(dim=1).to(device) * (~dones).to(device)
    else:
        targets = rewards.to(device) + discount_factor * maxQs.unsqueeze(dim=1).to(device) * (1 - dones).to(device)
    return targets
    
def train(Q, memory, optimizer, batch_size, discount_factor, target_net, device, error_clipping=True):
    # don't learn without some decent experience
    if len(memory) < batch_size:
        return None, None

    # random transition batch is taken from experience replay memory
    transitions = memory.sample(batch_size)
    
    # transition is a list of 4-tuples, instead we want 4 vectors (as torch.Tensor's)
    state, action, reward, next_state, done = zip(*transitions)
    
    # convert to PyTorch and define types
    state = torch.tensor(state, dtype=torch.float)
    action = torch.tensor(action, dtype=torch.int64)[:, None]  # Need 64 bit to use them as index
    next_state = torch.tensor(next_state, dtype=torch.float)
    reward = torch.tensor(reward, dtype=torch.float)[:, None]
    done = torch.tensor(done, dtype=torch.uint8)[:, None]  # Boolean
    
    # compute the q value
    q_val = compute_q_vals(Q, state, action, device)
    max_q = torch.max(torch.abs(q_val)).item()

    with torch.no_grad():  # Don't compute gradient info for the target (semi-gradient)
        target = compute_targets(Q, reward, next_state, done, discount_factor, target_net, device)
    
    # loss is measured from error between current and newly expected Q values
    loss = F.smooth_l1_loss(q_val, target)
    
    if error_clipping:
        loss = torch.clamp(loss, min=-1, max=1)

    # backpropagation of loss to Neural Network (PyTorch magic)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item(), max_q

In [6]:

def run_episodes(train, Q, policy, memory, env, num_episodes, batch_size, discount_factor, learn_rate, device, rendering_criterium=999999, use_target_net=10, error_clipping=True):
    optimizer = optim.Adam(Q.parameters(), learn_rate)
    if use_target_net:
        TARGET_UPDATE = use_target_net
        target_net = copy.deepcopy(Q)
        target_net.eval()
    else:
        target_net = Q
    
    global_steps = 0  # Count the steps (do not reset at episode start, to compute epsilon)
    episode_scores = []  #
    weights = []
    max_Qs = []
    rendering = False
    for i in tqdm(range(num_episodes)):
        state = env.reset()
        
        max_Qs_episode = []
        score = 0
        while True:
            policy.set_epsilon(get_epsilon(global_steps))
            action = policy.sample_action(state, device)
            state_next, reward, done, _ = env.step(action)                

            loss, max_Q_batch = train(Q, memory, optimizer, batch_size, discount_factor, target_net, device, error_clipping=error_clipping)

            max_Qs_episode.append(max_Q_batch)

            global_steps += 1
            score += reward

            if rendering:
                env.render()
                time.sleep(0.05)

            memory.push((state, action, reward, state_next, done))
            state = state_next

            if done:
                episode_scores.append(score)

                if all([q != None for q in max_Qs_episode]):
                    max_Qs.append(max(max_Qs_episode))
                else:
                    max_Qs.append(None)
                    
                weights.append(copy.deepcopy(Q).state_dict().copy())

                if score > rendering_criterium:
                    rendering = True
                else:
                    rendering = False
                break
                
            if use_target_net and (i % TARGET_UPDATE == 0):
                target_net.load_state_dict(Q.state_dict())
            
        env.close()

    return episode_scores, weights, max_Qs

In [7]:
with open('all_runs_weight_histoires.pkl', 'rb') as handle:
    weight_dict = pickle.load(handle)

weight_dict[0]["episode_network_weights"][0].keys()


odict_keys(['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias'])

In [110]:
cuda_ = "cuda:0"
device = torch.device(cuda_ if torch.cuda.is_available() else "cpu")

results = []
all_episode_weights = []

print(device)

# baseline tuning to ENV --------------------------------------------  

test_envs = ["CartPole-v1", "Acrobot-v1", "MountainCar-v0"]
runs_per_setup = 3
num_episodes = 1000

                # lr.          hidden size
# # Jan:        [1e-3], [256]:
# # Longxiang:  [1e-3], [128]:
# # Alex:       [1e-4], [256]:
# # Fabrizio:   [1e-4], [128]:

# Who are you?
name = "Jan"

# NO Tricks
for error_clipping in [False]:
    for use_target in [0]:       
        for memory_size in [64]:                   
    
            for env in test_envs:                                          # 3
                for i in range(runs_per_setup):                            # 3

                    # tuning these params
                    for discount_factor in [0.99, 0.8, 0.9]:               # 3       
                        for learn_rate in [1e-3]:                    # 2
                            for num_hidden in [256]:                  # 2
                                                                           # = 108 runs 
                                if memory_size < 64:
                                    batch_size = memory_size
                                else:
                                    batch_size = 64

                                seed = i # same seed for different setups, different for every run
                                new_env = gym.envs.make(env)
                                memory = ReplayMemory(memory_size)

                                random.seed(seed)
                                torch.manual_seed(seed)
                                new_env.seed(seed)

                                setup = {"env":env, "batch_size":batch_size, "use_target":use_target, "num_hidden":num_hidden, "learn_rate":learn_rate, "memory_size":memory_size, "discount_factor":discount_factor, "run":i}

                                try:
                                    # Continious obs space
                                    Q_net = QNetwork(num_hidden, observation_space=new_env.observation_space.low.shape[0], action_space=new_env.action_space.n).to(device)
                                    obs_space = new_env.observation_space.low.shape[0]
                                except:
                                    # Discrete obs space
                                    Q_net = QNetwork(num_hidden, observation_space=new_env.observation_space.n, action_space=new_env.action_space.n).to(device)
                                    obs_space = new_env.observation_space.n

                                policy = EpsilonGreedyPolicy(Q_net, 0.05)
                                episode_scores, episode_weights, max_Qs_episodes = run_episodes(train, Q_net, policy, memory, new_env, num_episodes, batch_size, discount_factor, learn_rate, device, use_target_net=use_target, error_clipping=error_clipping)        

                                # # L2 norm calculation
                                l2_norms=[]
                                for weights in episode_weights:
                                    all_weights = []
                                    for layer in weights.keys():
                                        all_weights.append(weights[layer].flatten())
                                    all_weights = torch.cat(all_weights, dim=0)
                                    l2_norms.append(torch.norm(((all_weights)), 2, -1))

                                # # Save results
                                results.append({**setup, **{"scores":episode_scores, "l2_norms":l2_norms, "maxQs": max_Qs_episodes, "soft_convergence_thresh":(1/(1-discount_factor))}})
                                df = pd.DataFrame(results)
                                df.to_csv(f"baseline_tune_results_{name}.csv")

                                # save network weight history
                                # all_episode_weights.append({**setup, **{"episode_network_weights":episode_weights}})
                                # with open('all_runs_weight_histoires.pkl', 'wb') as handle:
                                #     pickle.dump(all_episode_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

                                # PLOT SOFT CONVERGENCE DATA
                                # plt.plot(max_Qs_episodes)
                                # plt.plot([results[-1]["soft_convergence_thresh"]]*len(max_Qs_episodes))
                                # plt.show()



 14%|█▎        | 135/1000 [00:02<00:14, 61.33it/s]


KeyboardInterrupt: 

In [124]:
import glob
from collections import defaultdict

all_df = pd.DataFrame()
runs_per_setup = 3

print("Assure all files are here")
assert len(glob.glob("baseline*")) == 4
for fn in glob.glob("baseline*"):
    print(fn)
    df = pd.read_csv(fn)
    all_df = all_df.append(df, ignore_index=True)



scores_dict = defaultdict(float)
index_dict = defaultdict(float)
best_setup_score = defaultdict(lambda: float('-inf'))
best_setup_score_index = defaultdict(int)
best_setup = {}

for i, (scores, env, lr, hidden, discount) in enumerate(zip(all_df.scores, all_df.env, all_df.learn_rate, all_df.num_hidden, all_df.discount_factor)):
    max_score = max(eval(scores))
    max_score_index = np.argmax(eval(scores))

    scores_dict[f"{env}_{lr}_{hidden}_{discount}"] += (1/runs_per_setup) * max_score
    index_dict[f"{env}_{lr}_{hidden}_{discount}"] += (1/runs_per_setup) * max_score_index

for key, value in scores_dict.items():
    env, lr, hidden, discount = key.split("_")

    if value > best_setup_score[env]:
        best_setup_score[env] = value
        best_setup_score_index[env] = index_dict[f"{env}_{lr}_{hidden}_{discount}"]
        best_setup[env] = lr, hidden, discount

    elif value == best_setup_score[env]:
        if index_dict[f"{env}_{lr}_{hidden}_{discount}"] < best_setup_score_index[env]:
            best_setup_score[env] = value
            best_setup_score_index[env] = index_dict[f"{env}_{lr}_{hidden}_{discount}"]
            best_setup[env] = lr, hidden, discount
 
print(best_setup)


Assure all files are here
baseline_tune_results_longxiang.csv
baseline_tune_results_Alex.csv
baseline_tune_results_Jan.csv
baseline_tune_results_Fab.csv
{'CartPole-v1': ('0.001', '128', '0.8'), 'Acrobot-v1': ('0.001', '256', '0.99'), 'MountainCar-v0': ('0.0001', '128', '0.99')}


In [127]:
cuda_ = "cuda:0"
device = torch.device(cuda_ if torch.cuda.is_available() else "cpu")

results = []
all_episode_weights = []

print(device)

# Trick tuning --------------------------------------------  

test_envs = ["CartPole-v1", "Acrobot-v1", "MountainCar-v0"]
runs_per_setup = 3
num_episodes = 10

                # use_target
# # Jan:        [25]:
# # Longxiang:  [250]:
# # Alex:       [2500]:
# # Fabrizio:   [0]:

# Who are you?
name = "test"

for env in test_envs:                                               # 3
    # set best baseline paramaters per env
    learn_rate, num_hidden, discount_factor = best_setup[env]
    learn_rate = float(learn_rate)
    num_hidden = int(num_hidden)
    discount_factor = float(discount_factor)
    for i in range(runs_per_setup):                                 # 3
        
        for error_clipping in [False, True]:                        # 2
            for use_target in [0, 25, 250, 2500]:                   # 4
                for memory_size in [64, 5000, 10000, 20000]:        # 4
                                                                    # = 284 runs          
                                
                    if memory_size < 64:
                        batch_size = memory_size
                    else:
                        batch_size = 64

                    seed = i # same seed for different setups, different for every run
                    new_env = gym.envs.make(env)
                    memory = ReplayMemory(memory_size)

                    random.seed(seed)
                    torch.manual_seed(seed)
                    new_env.seed(seed)

                    setup = {"env":env, "batch_size":batch_size, "use_target":use_target, "num_hidden":num_hidden, "learn_rate":learn_rate, "memory_size":memory_size, "discount_factor":discount_factor, "run":i}

                    print(setup)

                    try:
                        # Continious obs space
                        Q_net = QNetwork(num_hidden, observation_space=int(new_env.observation_space.low.shape[0]), action_space=new_env.action_space.n).to(device)
                        obs_space = new_env.observation_space.low.shape[0]
                    except:
                    #     # Discrete obs space
                        Q_net = QNetwork(num_hidden, observation_space=new_env.observation_space.n, action_space=new_env.action_space.n).to(device)
                        obs_space = new_env.observation_space.n 
                      
                    policy = EpsilonGreedyPolicy(Q_net, 0.05)
                    episode_scores, episode_weights, max_Qs_episodes = run_episodes(train, Q_net, policy, memory, new_env, num_episodes, batch_size, discount_factor, learn_rate, device, use_target_net=use_target, error_clipping=error_clipping)        

                    # # L2 norm calculation
                    l2_norms=[]
                    for weights in episode_weights:
                        all_weights = []
                        for layer in weights.keys():
                            all_weights.append(weights[layer].flatten())
                        all_weights = torch.cat(all_weights, dim=0)
                        l2_norms.append(torch.norm(((all_weights)), 2, -1))

                    # # Save results
                    results.append({**setup, **{"scores":episode_scores, "l2_norms":l2_norms, "maxQs": max_Qs_episodes, "soft_convergence_thresh":(1/(1-discount_factor))}})
                    df = pd.DataFrame(results)
                    df.to_csv(f"trick_tune_results_{name}.csv")

                    # save network weight history
                    # all_episode_weights.append({**setup, **{"episode_network_weights":episode_weights}})
                    # with open('all_runs_weight_histoires.pkl', 'wb') as handle:
                    #     pickle.dump(all_episode_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

                    # PLOT SOFT CONVERGENCE DATA
                    # plt.plot(max_Qs_episodes)
                    # plt.plot([results[-1]["soft_convergence_thresh"]]*len(max_Qs_episodes))
                    # plt.show()





  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 99.39it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

cpu
{'env': 'CartPole-v1', 'batch_size': 64, 'use_target': 0, 'num_hidden': 128, 'learn_rate': 0.001, 'memory_size': 64, 'discount_factor': 0.8, 'run': 0}
{'env': 'CartPole-v1', 'batch_size': 64, 'use_target': 0, 'num_hidden': 128, 'learn_rate': 0.001, 'memory_size': 5000, 'discount_factor': 0.8, 'run': 0}
100%|██████████| 10/10 [00:00<00:00, 159.35it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:00<00:00, 130.01it/s]

  0%|          | 0/10 [00:00<?, ?it/s]{'env': 'CartPole-v1', 'batch_size': 64, 'use_target': 0, 'num_hidden': 128, 'learn_rate': 0.001, 'memory_size': 10000, 'discount_factor': 0.8, 'run': 0}
{'env': 'CartPole-v1', 'batch_size': 64, 'use_target': 0, 'num_hidden': 128, 'learn_rate': 0.001, 'memory_size': 20000, 'discount_factor': 0.8, 'run': 0}


100%|██████████| 10/10 [00:00<00:00, 85.52it/s]

  0%|          | 0/10 [0

KeyboardInterrupt: 

In [ ]:
cuda_ = "cuda:0"
device = torch.device(cuda_ if torch.cuda.is_available() else "cpu")

results = []
all_episode_weights = []

print(device)

# validation runs of best setups --------------------------------------------  

test_envs = ["CartPole-v1", "Acrobot-v1", "MountainCar-v0"]
runs_per_setup = 26
num_episodes = 1000


for env in test_envs:                                               # 3
    # set best baseline paramaters

    for i in range(runs_per_setup):                                 # 26
        
        for error_clipping in [False, True]:                        # 2
            for use_target in [0, best_target_refresh]:             # 2
                for memory_size in [64, best_memory_size]:          # 2
                                                                    # = 624 runs          
                                
                    if memory_size < 64:
                        batch_size = memory_size
                    else:
                        batch_size = 64

                    seed = i # same seed for different setups, different for every run
                    new_env = gym.envs.make(env)
                    memory = ReplayMemory(memory_size)

                    random.seed(seed)
                    torch.manual_seed(seed)
                    new_env.seed(seed)

                    setup = {"env":env, "batch_size":batch_size, "use_target":use_target, "num_hidden":num_hidden, "learn_rate":learn_rate, "memory_size":memory_size, "discount_factor":discount_factor, "run":i}

                    try:
                        # Continious obs space
                        Q_net = QNetwork(num_hidden, observation_space=new_env.observation_space.low.shape[0], action_space=new_env.action_space.n).to(device)
                        obs_space = new_env.observation_space.low.shape[0]
                    except:
                        # Discrete obs space
                        Q_net = QNetwork(num_hidden, observation_space=new_env.observation_space.n, action_space=new_env.action_space.n).to(device)
                        obs_space = new_env.observation_space.n

                    policy = EpsilonGreedyPolicy(Q_net, 0.05)
                    episode_scores, episode_weights, max_Qs_episodes = run_episodes(train, Q_net, policy, memory, new_env, num_episodes, batch_size, discount_factor, learn_rate, device, use_target_net=use_target, error_clipping=error_clipping)        

                    # # L2 norm calculation
                    l2_norms=[]
                    for weights in episode_weights:
                        all_weights = []
                        for layer in weights.keys():
                            all_weights.append(weights[layer].flatten())
                        all_weights = torch.cat(all_weights, dim=0)
                        l2_norms.append(torch.norm(((all_weights)), 2, -1))

                    # # Save results
                    results.append({**setup, **{"scores":episode_scores, "l2_norms":l2_norms, "maxQs": max_Qs_episodes, "soft_convergence_thresh":(1/(1-discount_factor))}})
                    df = pd.DataFrame(results)
                    df.to_csv("validation_results.csv")

                    # save network weight history
                    # all_episode_weights.append({**setup, **{"episode_network_weights":episode_weights}})
                    # with open('all_runs_weight_histoires.pkl', 'wb') as handle:
                    #     pickle.dump(all_episode_weights, handle, protocol=pickle.HIGHEST_PROTOCOL)

                    # PLOT SOFT CONVERGENCE DATA
                    # plt.plot(max_Qs_episodes)
                    # plt.plot([results[-1]["soft_convergence_thresh"]]*len(max_Qs_episodes))
                    # plt.show()

